In [128]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import folium
import random

In [129]:
# Cargar el archivo CSV descargado
df = pd.read_csv('/content/2023_For_Hire_Vehicles_Trip_Data_20241021.csv')

# Verifica las primeras filas del dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 176354 entries, 0 to 176353
Data columns (total 7 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   dispatching_base_num    176354 non-null  object 
 1   pickup_datetime         176354 non-null  object 
 2   dropOff_datetime        176354 non-null  object 
 3   PUlocationID            34989 non-null   float64
 4   DOlocationID            148788 non-null  float64
 5   SR_Flag                 0 non-null       float64
 6   Affiliated_base_number  168174 non-null  object 
dtypes: float64(3), object(4)
memory usage: 9.4+ MB


In [130]:
# Convertir las columnas de tiempo al formato datetime especificando el formato adecuado
df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'], format='%m/%d/%Y %I:%M:%S %p', errors='coerce')
df['dropOff_datetime'] = pd.to_datetime(df['dropOff_datetime'], format='%m/%d/%Y %I:%M:%S %p', errors='coerce')

# Calcular el tiempo total de viaje
df['trip_duration'] = (df['dropOff_datetime'] - df['pickup_datetime']).dt.total_seconds()

# Explorar los datos preprocesados
print(df[['pickup_datetime', 'dropOff_datetime', 'trip_duration']].head())




      pickup_datetime    dropOff_datetime  trip_duration
0 2023-03-01 00:24:00 2023-03-01 02:35:00         7860.0
1 2023-01-01 00:30:00 2023-01-01 01:00:00         1800.0
2 2023-01-01 00:01:00 2023-01-01 03:15:00        11640.0
3 2023-01-01 00:30:00 2023-01-01 01:05:00         2100.0
4 2023-01-01 00:34:45 2023-01-01 00:52:03         1038.0


In [131]:
# Eliminar todas las filas donde PUlocationID es nulo
df_cleaned = df.dropna(subset=['PUlocationID'])
df_cleaned = df_cleaned.dropna(subset=['DOlocationID'])


In [132]:
# Resetear los índices del DataFrame
df_cleaned = df_cleaned.reset_index(drop=True)

# Eliminar la columna 'SR_Flag'
df_cleaned = df_cleaned.drop(columns=['SR_Flag'])

# Verificar que los cambios se aplicaron correctamente
df_cleaned


,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,Affiliated_base_number,trip_duration
0,B00254,2023-01-01 00:00:20,2023-01-01 00:38:41,132.0,265.0,B00254,2301.0
1,B00254,2023-01-01 00:30:10,2023-01-01 00:48:51,65.0,158.0,B03404,1121.0
2,B00445,2023-01-01 00:45:35,2023-01-01 01:04:26,95.0,15.0,B00445,1131.0
3,B00446,2023-01-01 00:09:48,2023-01-01 00:29:53,236.0,234.0,B00446,1205.0
4,B00706,2023-01-01 00:45:31,2023-01-01 00:50:32,206.0,245.0,B00706,301.0
...,...,...,...,...,...,...,...
34950,B02532,2023-01-06 09:00:00,2023-01-06 09:31:00,258.0,92.0,B02532,1860.0
34951,B02532,2023-01-06 09:00:00,2023-01-06 11:02:00,75.0,92.0,B02532,7320.0
34952,B02532,2023-01-06 09:10:00,2023-01-06 09:58:00,258.0,265.0,B02532,2880.0
34953,B02532,2023-01-06 09:35:00,2023-01-06 10:07:00,218.0,265.0,B02532,1920.0


In [133]:
taxi_zones = pd.read_excel('/content/codes.xlsx')
taxi_zones.set_index('LocationID', inplace=True)
taxi_zones

,Borough,Zone,Latitude,Longitude
LocationID,,,,
1,EWR,Newark Airport,40.6895,-74.1745
2,Queens,Jamaica Bay,40.6000,-73.8000
3,Bronx,Allerton/Pelham Gardens,40.8500,-73.8600
4,Manhattan,Alphabet City,40.7255,-73.9851
5,Staten Island,Arden Heights,40.5700,-74.1500
...,...,...,...,...
259,Bronx,Woodlawn/Wakefield,40.8913,-73.8570
260,Queens,Woodside,40.7471,-73.9067
261,Manhattan,World Trade Center,40.7127,-74.0134


In [134]:

taxi_data = df_cleaned
location_data = taxi_zones

In [135]:
def calculate_distance_matrix(locations):
    """
    Calcula una matriz de distancias entre todas las ubicaciones.

    Args:
        locations (DataFrame): Un DataFrame que contiene las coordenadas de las ubicaciones con columnas 'Latitude' y 'Longitude'.

    Returns:
        ndarray: Matriz de distancias euclidianas entre las ubicaciones.
    """
    latitudes = locations['Latitude'].values
    longitudes = locations['Longitude'].values
    return np.sqrt((latitudes[:, np.newaxis] - latitudes[np.newaxis, :]) ** 2 +
                   (longitudes[:, np.newaxis] - longitudes[np.newaxis, :]) ** 2)
# Crear matriz de distancias
distance_matrix = calculate_distance_matrix(location_data)


In [136]:
def calculate_route_distance(route):
    """
    Calcula la distancia total de una ruta dada.

    Args:
        route (list): Una lista de índices que representan la ruta.

    Returns:
        float: La distancia total de la ruta.
    """
    return sum(distance_matrix[route[i], route[i + 1]] for i in range(len(route) - 1))

In [137]:
def initialize_penguins(num_penguins, start_index, end_index):
    """
    Inicializa una lista de 'pingüinos' (soluciones) con posiciones aleatorias.

    Args:
        num_penguins (int): El número de pingüinos a inicializar.
        start_index (int): El índice de la ubicación inicial.
        end_index (int): El índice de la ubicación final.

    Returns:
        list: Una lista de rutas inicializadas.
    """
    return [
        [start_index] + random.sample(range(num_locations), num_locations - 2) + [end_index]
        for _ in range(num_penguins)
    ]

In [138]:
def update_penguins(penguins, best_route):
    """
    Actualiza las rutas de los pingüinos según su aptitud comparada con la mejor ruta.

    Args:
        penguins (list): Lista de rutas actuales de los pingüinos.
        best_route (list): La mejor ruta encontrada hasta ahora.

    Returns:
        list: Lista actualizada de rutas de los pingüinos.
    """
    new_penguins = []
    for penguin in penguins:
        if calculate_route_distance(penguin) < calculate_route_distance(best_route):
            new_penguins.append(penguin)
        else:
            new_route = penguin[:-1]  # Excluir el índice final
            random.shuffle(new_route)
            new_penguins.append([penguin[0]] + new_route + [penguin[-1]])
    return new_penguins

In [139]:
def pesoa_optimization(num_iterations, num_penguins, start_index, end_index):
    """
    Ejecuta el algoritmo de optimización de PeSOA para encontrar la mejor ruta.

    Args:
        num_iterations (int): Número de iteraciones para el algoritmo.
        num_penguins (int): Número de pingüinos (soluciones) en la población.
        start_index (int): Índice de la ubicación inicial.
        end_index (int): Índice de la ubicación final.

    Returns:
        tuple: La mejor ruta encontrada y su distancia total.
    """
    penguins = initialize_penguins(num_penguins, start_index, end_index)
    best_route = min(penguins, key=calculate_route_distance)

    for _ in range(num_iterations):
        penguins = update_penguins(penguins, best_route)
        current_best = min(penguins, key=calculate_route_distance)
        if calculate_route_distance(current_best) < calculate_route_distance(best_route):
            best_route = current_best

    return best_route, calculate_route_distance(best_route)


In [140]:
# Parámetros
num_iterations = 1000
num_penguins = 100
num_locations = len(location_data)

# Especifica el índice de la ubicación inicial y final
start_index = 1
end_index = 4

# Ejecución del algoritmo
best_route, best_distance = pesoa_optimization(num_iterations, num_penguins, start_index, end_index)

# Mostrar resultados
print("Mejor ruta encontrada:", best_route)
print("Distancia total de la mejor ruta:", best_distance)

# Mostrar el mapa con la mejor ruta
ny_map = folium.Map(location=[40.7128, -74.0060], zoom_start=11)

# Agregar marcadores para cada ubicación de la mejor ruta
for index in best_route:
    lat = location_data.iloc[index]['Latitude']
    lon = location_data.iloc[index]['Longitude']
    folium.Marker(
        location=[lat, lon],
        popup=location_data.iloc[index]['Zone'],
        icon=folium.Icon(color='blue')
    ).add_to(ny_map)

# Trazar la mejor ruta en el mapa
route_coordinates = [
    (location_data.iloc[index]['Latitude'], location_data.iloc[index]['Longitude'])
    for index in best_route
]
folium.PolyLine(route_coordinates, color='red', weight=2.5, opacity=1).add_to(ny_map)

# Mostrar el mapa en el notebook
display(ny_map)


# Crear un mapa centrado en Nueva York
ny_map = folium.Map(location=[40.7128, -74.0060], zoom_start=11)

# Agregar marcadores para cada zona
for _, row in taxi_zones.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=row['Zone'],
        icon=folium.Icon(color='blue')
    ).add_to(ny_map)

# Guardar el mapa en un archivo HTML
ny_map.save('taxi_zones_map.html')

Mejor ruta encontrada: [1, 177, 63, 9, 4, 23, 197, 127, 3, 244, 55, 99, 156, 111, 104, 204, 1, 123, 260, 192, 32, 139, 254, 219, 198, 27, 183, 253, 112, 231, 51, 117, 108, 211, 152, 174, 161, 223, 115, 164, 91, 5, 67, 168, 158, 76, 103, 194, 250, 25, 21, 61, 39, 196, 248, 163, 142, 203, 22, 261, 214, 180, 26, 210, 216, 137, 179, 47, 235, 212, 87, 233, 105, 74, 119, 132, 247, 73, 208, 41, 15, 14, 48, 234, 205, 71, 259, 148, 1, 16, 200, 1, 202, 80, 173, 191, 101, 65, 232, 136, 225, 201, 172, 257, 82, 206, 106, 35, 81, 54, 181, 6, 122, 107, 209, 7, 189, 36, 170, 116, 140, 121, 151, 190, 72, 162, 90, 120, 226, 37, 240, 237, 95, 193, 70, 227, 88, 252, 220, 45, 239, 59, 215, 228, 185, 42, 30, 243, 125, 224, 109, 229, 118, 251, 230, 130, 18, 0, 188, 184, 124, 178, 13, 207, 129, 64, 43, 20, 85, 133, 11, 187, 147, 1, 199, 182, 110, 167, 255, 221, 62, 256, 141, 246, 242, 79, 153, 159, 135, 241, 114, 100, 165, 146, 8, 160, 28, 34, 2, 94, 149, 134, 166, 262, 31, 113, 213, 238, 102, 12, 44, 24, 10,

In [141]:
# Asegúrate de que taxi_zones tenga 'LocationID' como columna normal
taxi_zones.reset_index(inplace=True)

# Ahora puedes vincular zonas de recogida y entrega
taxi_data = taxi_data.merge(taxi_zones[['LocationID', 'Zone']], left_on='PUlocationID', right_on='LocationID', how='left')
taxi_data = taxi_data.rename(columns={'Zone': 'PickupZone'})

taxi_data = taxi_data.merge(taxi_zones[['LocationID', 'Zone']], left_on='DOlocationID', right_on='LocationID', how='left')
taxi_data = taxi_data.rename(columns={'Zone': 'DropOffZone'})
# Calcular distancia promedio por zona de recogida
avg_trip_duration_by_zone = taxi_data.groupby('PickupZone')['trip_duration'].mean().reset_index()
print(avg_trip_duration_by_zone)

                  PickupZone  trip_duration
0    Allerton/Pelham Gardens    2252.582090
1              Alphabet City    1289.851064
2              Arden Heights    1480.104167
3    Arrochar/Fort Wadsworth    1344.137778
4                    Astoria     840.604938
..                       ...            ...
249       Woodlawn/Wakefield    1595.070588
250                 Woodside    1205.060345
251       World Trade Center    2177.693333
252           Yorkville East    1574.594203
253           Yorkville West    1272.748031

[254 rows x 2 columns]
